In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
spark = SparkSession.builder \
    .appName("MyApp") \
    .config("spark.driver.extraClassPath", "C:/Program Files (x86)/PostgreSQL/pgJDBC/postgresql-42.5.1.jar") \
    .getOrCreate()

In [17]:
jdbc_url = "jdbc:postgresql://localhost:5432/clientordersetl"
connection_properties = {
    "user": "postgres",
    "password": "123456",
    "driver": "org.postgresql.Driver"
}

In [6]:
dfclients = spark.read.format("csv").option("header","true").load("data/client.csv")
dforders = spark.read.format("csv").option("header","true").load("data/orders.csv")

In [10]:
dfjoined = dforders.join(dfclients,"client_id","inner")

In [19]:
dfresult = dfjoined.groupBy("client_id").agg(
    F.max("client_name").alias("ClientName"),
    F.round(F.avg("order_amount"),2).alias("AvgAmount"),
    F.sum("order_amount").alias("SumAmount"),
    F.count("client_id").alias("CountOrders")
)

In [20]:
dfresult.write.jdbc(url = jdbc_url,\
                    table='public."aggregated_results"',\
                    mode="overwrite",\
                    properties= connection_properties)